Name(s): John Michael Tran

# Association Analysis

Association analysis uses machine learning to extract frequent itemsets and strong association rules from large datasets. In this assignment you'll be implementing one of the most commonly used algorithms for association rule mining - the Apriori algorithm.

The dataset (`large_retail.txt`) that we are going to use has been adapted from the [Retail Market Basket Dataset](http://fimi.ua.ac.be/data/retail.pdf). This dataset contains transaction records supplied by a Belgian retail supermarket store. Each line in the file represents a separate transaction with the item ids separated by space. The dataset has 3000 transactions and 99 different item ids.

You are also provided with a smaller dataset (`small_retail.txt`) with 9 transactions and 5 different item ids along with the solutions. You can test and debug your implementation on this smaller dataset.

## Apriori Algorithm from scratch

The Apriori algorithm is a classical algorithm in data mining. It is used for mining frequent itemsets and relevant association rules. In this part, you'll be implementing this algorithm for generating the itemsets that occur enough times to meet the `min_sup` threshold.

**Implementation Hint:**

- Use the `frozenset` data structure in Python, which is similar to `set` in functionality, to represent the itemsets, because `frozenset` is an immutable (hashable) data structure. You can maintain a dictionary that maps from the itemset (a `frozenset`) to its support count.

In [75]:
# Standard imports (you can add additional headers if you wish)
import numpy as np

In [76]:
# Reading the dataset from file
def load_dataset(file_name):
    with open(file_name, 'r') as f:
        content = f.readlines()
        data = [[int(x) for x in line.rstrip().split()] for line in content]
    return data

In [77]:
# Read in the small_retail dataset
small_dataset = load_dataset('small_retail.txt')
small_dataset

[[1, 2, 5],
 [2, 4],
 [2, 3],
 [1, 2, 4],
 [1, 3],
 [2, 3],
 [1, 3],
 [1, 2, 3, 5],
 [1, 2, 3]]

**Q1.** Implement the function `create_1_itemsets` that takes as input the entire dataset and returns a list of all the 1-itemsets. For example, for `small_retail.txt` it should return:
~~~
[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]
 ~~~
 Please **don't hardcode** the item ids, your code should support item ids that are non-sequential.

In [78]:
def create_1_itemsets(dataset):
    c1 = []
    for dp in dataset :
        for item in dp :
            one_item = frozenset([item])
            if one_item not in c1 :
                c1.append(one_item)
    return c1


**Q2.** Implement function `filter_candidates` that takes as input the candidate itemsets, the dataset, and the minumum support count `min_sup`, and filters out candidates that don't meet the support threshold.

You should also return the support count information as a `dict` for the itemsets that meet `min_sup`. 

In [79]:
def filter_candidates(candidates, dataset, min_sup):
    retlist = []
    support_data = {}
    counts = {}
    for transaction in dataset :
        for itemset in candidates :
            if itemset.issubset(transaction) :
                if itemset in counts :
                    counts[itemset] += 1
                else :
                    counts[itemset] = 1
    
    for itemset in counts :
        if counts[itemset] >= min_sup :
            retlist.append(itemset)
            support_data[itemset] = counts[itemset]
 

    sortlist = [list(sets) for sets in retlist]
    sortlist.sort()
    retlist = [frozenset(sort) for sort in sortlist]
    return retlist, support_data

**Q3.** Implement the function `generate_next_itemsets` that takes in frequent itemsets of size `k` and generates candidate itemsets of size `k + 1`.

Use either the F(k-1) x F(k-1) or the F(k-1) x F(1) candidate generation method, then **filter the candidate list based on the apriori principle before returning it.**

In [80]:
def generate_next_itemsets(freq_sets):
    retList = []
    for i in range(len(freq_sets)) :
        for j in range(i+1, len(freq_sets)) :
            curr_set = freq_sets[i]
            next_set = freq_sets[j]
            curr_list = list(curr_set)
            next_list = list(next_set)
            curr_list.sort()
            next_list.sort()
            if curr_list[:-1]  == next_list[:-1] : 
                merge_set = curr_set.union(next_set)
                retList.append(merge_set)

    return retList
    

**Q4.** Implement the function `apriori_freq_itemsets` that takes the entire dataset as input and returns all the frequent itemsets that meet `min_sup`.

In [81]:

def apriori_freq_itemsets(dataset, minsup):
    c1 = create_1_itemsets(dataset)
    freq_list, freq_counts = filter_candidates(c1, dataset, minsup)
    retlist = [freq_list]
    index = 0
    while len(retlist[index]) > 0 :
        itemset = retlist[index]
        n_itemset = generate_next_itemsets(itemset)
        n_freq_list, n_freq_counts = filter_candidates(n_itemset, dataset, minsup)
        retlist.append(n_freq_list)
        freq_counts.update(n_freq_counts)
        index+=1
    return retlist, freq_counts

def print_apriori_table(items, freq_counts,dataset) : 
    print("Sup\tFreq Itemset")
    for item in items :
        for i in item :
            print(str(round(freq_counts[i] / len(dataset),2)) + "\t" + str(list(i)))

**Q5.** Display the frequent item sets in the form of a table along with their `support` (as a fraction: support count over number of transactions) for the `large_retail.txt` dataset **with a min support count of 300**.

Sample Table Format (tab separated table)

~~~
Sup     Freq Itemset
0.67	[1]
0.44	[1, 2]
(and so on)
...
...
~~~

`support(itemset) = support_count(itemset) / num_total_transactions`.

The `support` and the itemset should be separated by a tab (`'\t'`).

Note that the `support` should be rounded to the nearest 2 decimal places (use `round(sup, 2)`). If a support_fraction only contains 1 decimal place (for example, 0.1), you do not need to add a 0 to the end of it (leaving it as 0.1 is fine).

The itemsets should also be in a sorted order where smaller itemsets should come before larger itemsets and itemsets of the same size should be sorted amongst themselves.

For eg. 
~~~~
[1, 2] should come before [1, 2, 3]
[1, 2, 3] should come before [1, 2, 4]
[1, 2, 3] should come before [1, 4, 5]
[1, 2, 3] should come before [2, 3, 4]
~~~~

Note that **this order is very important for grading!** 

The output also **shouldn't contain any duplicates**. 

The sample output for the `small_retail.txt` dataset with `min_sup` set to 2 is:

~~~~
Sup     Freq Itemset
0.67    [1]
0.78	[2]
0.67	[3]
0.22	[4]
0.22	[5]
0.44	[1, 2]
0.44	[1, 3]
0.22	[1, 5]
0.44	[2, 3]
0.22	[2, 4]
0.22	[2, 5]
0.22	[1, 2, 3]
0.22	[1, 2, 5]
~~~~


In [82]:
large_dataset = load_dataset('large_retail.txt')
items, freq_counts = apriori_freq_itemsets(large_dataset, 300)
print_apriori_table(items, freq_counts, large_dataset)

Sup	Freq Itemset
0.1	[31]
0.14	[32]
0.11	[36]
0.26	[38]
0.53	[39]
0.22	[41]
0.47	[48]
0.11	[60]
0.11	[65]
0.11	[89]
0.14	[32, 39]
0.15	[38, 39]
0.14	[41, 39]
0.13	[48, 38]
0.33	[48, 39]
0.18	[48, 41]
0.14	[48, 41, 39]
